In [118]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [127]:
df = pd.read_pickle('BeatlesSongsClustered.pkl')

##These facts are sorted in alphabetical order. To elminiate any potiential bias, I will shuffle the order of the facts

#I know I don't have to do this, but I'm going to do it anyway. old instincts when we had to do all of this without fancy libraries

df['random'] = np.random.rand(len(df))
df.sort_values('random', inplace=True)
df.drop('random', axis=1, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df.dtypes
             

Id                                      int64
SongKey                                object
song                                   object
songwriters                            object
lead_vocals                            object
year                                    int32
cover                                   int64
Era                                    object
first_release_date             datetime64[ns]
max_key                               float64
mode                                  float64
avg_danceability                      float64
avg_energy                            float64
avg_loudness                          float64
avg_speechiness                       float64
avg_acousticness                      float64
avg_instrumentalness                  float64
avg_liveness                          float64
avg_valence                           float64
avg_tempo                             float64
avg_duration_ms                       float64
avg_time_signature                

In [128]:


# Selecting only numeric features
num_features = ['year', 'avg_danceability', 'avg_energy', 'avg_loudness', 'avg_speechiness', 
                'avg_acousticness', 'avg_instrumentalness', 'avg_liveness', 'avg_valence', 
                'avg_tempo', 'avg_duration_ms', 'avg_time_signature', 'avg_popularity'
                ]


cat_features = ['Era', 'mode', 'keyname', 'Cluster_Name']
                

##This is just a handy reference or could be useful if I want to do some more feature engineering
excluded_features = ['Id', 'SongKey', 'song', 'songwriters', 'lead_vocals', 'cover',
                     'first_release_date', 'max_key', 'Song_y', 'lyrics', 
                     'core_catalogue_releases', 'Canonical_album', 'first_vocalist',
                     ###'Paul_Song', 
                     'John_Song', 'George_Song', 'Ringo_Song', 'popularity', 
                     'BB_name', 'BB_artist', 'BB_debut_date', 'BB_peak_date', 
                     'distilbertsentiment', 'FinetunedSentiment_Negative', 
                     'FinetunedSentiment_Positive', 'cluster', 'vader_neg', 'vader_neu',
                     'vader_pos', 'vader_compound', 'roberta_neg', 'roberta_neu', 
                     'roberta_pos', 'Pipeline_sentiment_label', 'Pipeline_sentiment_score']

In [129]:
## Try Decision Tree Classifier

from sklearn.preprocessing import OneHotEncoder

# Define Preprocessing for numeric columns (scale them)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Define Preprocessing for categorical columns (encode them)
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])  # We use OneHotEncoder here

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)])

# Fill missing values with the median of the column
df = df.fillna(df.median())

# Split data into features and target variable
X = df.drop(columns='Paul_Song')
y = df['Paul_Song']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create a pipeline that does the preprocessing and then fits the classifier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

# Train the classifier
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))

#Confusion Matrix
print("Confusion Matrix:",confusion_matrix(y_test, y_pred))


##Accuracy: 0.5
##Confusion 
# Matrix: [[15  9]
#          [11  5]]





Accuracy: 0.625
Confusion Matrix: [[20  8]
 [ 7  5]]


C:\Users\rob\AppData\Local\Temp\ipykernel_20420\1446395236.py:20: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df = df.fillna(df.median())
C:\Users\rob\AppData\Local\Temp\ipykernel_20420\1446395236.py:20: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.median())


# Not a good canidate for tree classification because it won't generalize. Overfitting

## Try Random Forest

In [130]:
from sklearn.ensemble import RandomForestClassifier

# Define Preprocessing for numeric columns (scale them)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Define Preprocessing for categorical columns (encode them)
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])  # We use OneHotEncoder here

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)])

# Fill missing values with the median of the column
df = df.fillna(df.median())

# Split data into features and target variable
X = df.drop(columns='Paul_Song')
y = df['Paul_Song']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create a pipeline that does the preprocessing and then fits the classifier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())]) # We use RandomForestClassifier here

# Train the classifier
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))

#Confusion Matrix
print("Confusion Matrix:",confusion_matrix(y_test, y_pred))
# Precision, Recall, F1 Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))



## Accuracy: 0.775
## Confusion Matrix: [[28  0]
 #                    [ 9  3]]


Accuracy: 0.75
Confusion Matrix: [[27  1]
 [ 9  3]]
Precision: 0.75
Recall: 0.25
F1 Score: 0.38


C:\Users\rob\AppData\Local\Temp\ipykernel_20420\1383498590.py:18: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df = df.fillna(df.median())
C:\Users\rob\AppData\Local\Temp\ipykernel_20420\1383498590.py:18: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.median())


In [131]:
# Get feature importances
importances = clf.named_steps['classifier'].feature_importances_

# Get feature names
num_features_names = num_features
cat_features_names = clf.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['encoder'].get_feature_names_out(cat_features)

features_names = np.concatenate([num_features_names, cat_features_names])

# Now that you have the importances and corresponding feature names,
# you can sort the features by importance

feature_importances = pd.DataFrame({'feature': features_names, 'importance': importances})
feature_importances.sort_values(by='importance', ascending=False, inplace=True)

print(feature_importances)


                                feature  importance
5                      avg_acousticness    0.113039
1                      avg_danceability    0.090280
2                            avg_energy    0.080349
4                       avg_speechiness    0.072814
7                          avg_liveness    0.068528
12                       avg_popularity    0.065799
10                      avg_duration_ms    0.062332
8                           avg_valence    0.057789
6                  avg_instrumentalness    0.056696
3                          avg_loudness    0.054504
9                             avg_tempo    0.054486
0                                  year    0.042008
26                            keyname_F    0.024341
11                   avg_time_signature    0.015815
13                      Era_Beatlemania    0.010811
31                  Cluster_Name_Angsty    0.010142
28                            keyname_G    0.009910
18                            keyname_A    0.009424
33    Cluste